In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import os
os.listdir('/content')

['.config', 'sample_data', 'drive']

In [15]:
import shutil

# Copy the diabetes.csv file from the temporary location to your Google Drive
shutil.copy('/diabetes.csv', '/content/drive/MyDrive/diabetes.csv')
print("File copied successfully to Google Drive!")

File copied successfully to Google Drive!


In [16]:
import pandas as pd

# Now, load the CSV file from Google Drive into a pandas DataFrame
df = pd.read_csv('/content/drive/MyDrive/diabetes.csv')

# Display the first 5 rows of the DataFrame to confirm it loaded correctly
display(df.head())

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [17]:
# Save the DataFrame to a CSV file in Google Drive
df.to_csv('/content/drive/MyDrive/processed_diabetes.csv', index=False)
print("DataFrame saved successfully to '/content/drive/MyDrive/processed_diabetes.csv'")

DataFrame saved successfully to '/content/drive/MyDrive/processed_diabetes.csv'


In [18]:
# Display descriptive statistics of the DataFrame
display(df.describe())

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


### Handling Implausible Zero Values

Several columns ('Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI') show a minimum value of 0, which is biologically implausible for these measurements. We will treat these 0s as missing values and replace them with `NaN` (Not a Number).

In [19]:
import numpy as np

# Columns identified with implausible zero values
columns_with_zeros = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

# Replace 0s with NaN in the identified columns
df[columns_with_zeros] = df[columns_with_zeros].replace(0, np.nan)

print("0s replaced with NaN in specified columns.")

# Display the count of NaN values in these columns to verify
print("\nNaN counts after replacement:")
display(df[columns_with_zeros].isnull().sum().to_frame('NaN Count'))

0s replaced with NaN in specified columns.

NaN counts after replacement:


,NaN Count
Glucose,5
BloodPressure,35
SkinThickness,227
Insulin,374
BMI,11


### Imputing Missing Values with the Median

Now that the implausible 0s are marked as `NaN`, we will impute these missing values using the median of each respective column. The median is chosen over the mean as it is more robust to outliers.

In [21]:
# Impute missing values with the median of each column
for column in columns_with_zeros:
    median_val = df[column].median()
    df[column] = df[column].fillna(median_val)

print("Missing values imputed with median for specified columns.")

# Verify that there are no more NaN values in these columns
print("\nNaN counts after imputation:")
display(df[columns_with_zeros].isnull().sum().to_frame('NaN Count'))

# Display descriptive statistics again to see the impact of imputation
print("\nDescriptive statistics after imputation:")
display(df.describe())

Missing values imputed with median for specified columns.

NaN counts after imputation:


,NaN Count
Glucose,0
BloodPressure,0
SkinThickness,0
Insulin,0
BMI,0



Descriptive statistics after imputation:


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,121.656250,72.386719,29.108073,140.671875,32.455208,0.471876,33.240885,0.348958
std,3.369578,30.438286,12.096642,8.791221,86.383060,6.875177,0.331329,11.760232,0.476951
min,0.000000,44.000000,24.000000,7.000000,14.000000,18.200000,0.078000,21.000000,0.000000
25%,1.000000,99.750000,64.000000,25.000000,121.500000,27.500000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,29.000000,125.000000,32.300000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [22]:
from re import X
# Features (input)
X = df.drop('Outcome', axis=1)

# Target (output)
y = df['Outcome']

print("X shape:", X.shape)
print("y shape:", y.shape)

X shape: (768, 8)
y shape: (768,)


In [25]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Training data:", X_train.shape, y_train.shape)
print("Testing data:", X_test.shape, y_test.shape)


Training data: (614, 8) (614,)
Testing data: (154, 8) (154,)


In [27]:
from sklearn.linear_model import LogisticRegression

In [28]:
model = LogisticRegression(max_iter=1000)


In [29]:
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [30]:
y_pred = model.predict(X_test)

In [31]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print("Model Accuracy:", accuracy)

Model Accuracy: 0.7532467532467533


In [34]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)
print(cm)

[[82 17]
 [21 34]]


In [35]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.83      0.81        99
           1       0.67      0.62      0.64        55

    accuracy                           0.75       154
   macro avg       0.73      0.72      0.73       154
weighted avg       0.75      0.75      0.75       154

